En este docuemnto realizamos la carga de los datos y realizamos un análisis general de los distintos años. Revisaremos que todas las columnas contengan los datos correctos, que los tipos de dato sean correctos, realizaremos un tratamiento de los valores nulos y de los duplicados. Finalmente realizaremos la unión de todos los datos.

In [28]:
# Importamos las librerías a utilizar
import pandas as pd
import numpy as np

In [29]:
# Indicamos que queremos que nos enseñe por pantalla todas las columnas de los DataFrame que vamos a cargar
pd.set_option('display.max_columns', None)

Ponemos las funciones que usaremos en el análisis

In [30]:
def info_df(df):
    """
    Función que devuelve información general sobre el DatFrame que le pasemos.


    Args:
        df (DataFrame): DataFrame con información que queramos revisar

    Returns:
        DataFrame: DataFrame con información general sobre las columnas del DataFrame que se le ha pasado a la función: tipo de datos, número de
        registros, número de valores nulos, porcentaje de los valores nulos sobre el total
    """
    info_df = pd.DataFrame()
    info_df["Tipo_dato"] = df.dtypes
    info_df["numero_registros"] = [df[elemento].value_counts().sum() for elemento in df]
    info_df["Numero_nulos"] = round(df.isnull().sum())
    info_df["%_nulos"] = round((df.isnull().sum()/df.shape[0])*100, 2)

    return info_df

# Análisis año a año del desarrollo económico de Brasil

**Datos 2013**

In [31]:
# Nos ha surgido un problema a la hora de cargar los datos debido a que los datos no están en español, sino en portugués (son datos de Brasil). 
datos_2013 = pd.read_csv("datos/datos-2013.csv", sep = ";", encoding = "latin-1")
datos_2013. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,"0,00","0,00","1297,13","0,00",31/12/2013,2013


In [32]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2013 = datos_2013.copy()
copia_datos_2013.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,"0,00","0,00","1297,13","0,00",31/12/2013,2013


Información general del DataFrame de 2013

In [33]:
info_df(datos_2013)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,4498,0,0.0
NOME ÓRGÃO SUPERIOR,object,4498,0,0.0
CÓDIGO ÓRGÃO,int64,4498,0,0.0
NOME ÓRGÃO,object,4498,0,0.0
CÓDIGO UNIDADE GESTORA,int64,4498,0,0.0
NOME UNIDADE GESTORA,object,4498,0,0.0
CATEGORIA ECONÔMICA,object,4498,0,0.0
ORIGEM RECEITA,object,4498,0,0.0
ESPÉCIE RECEITA,object,4498,0,0.0
DETALHAMENTO,object,4498,0,0.0


In [34]:
# Nos encontramos con varias columnas que contienen datos númericos que tienen formato objeto. 

print(datos_2013["VALOR PREVISTO ATUALIZADO"].dtypes)
print(datos_2013["VALOR LANÇADO"].dtypes)
print(datos_2013["VALOR REALIZADO"].dtypes)
print(datos_2013["PERCENTUAL REALIZADO"].dtypes)

# Para poder trabajar con estas columnas vamos a necesitar cambiar el tipo de dato de las mismas.

object
object
object
object


In [35]:
# Como queremos modificar las columnas anteriores, nos creamos una lista que contenga las mismas. Para ello hacemos uso de iloc. 
columnas_2013 = datos_2013.iloc[:,10: 14].columns
columnas_2013

Index(['VALOR PREVISTO ATUALIZADO', 'VALOR LANÇADO', 'VALOR REALIZADO',
       'PERCENTUAL REALIZADO'],
      dtype='object')

In [36]:
# Para poder realizar el cambio de estas columnas del tipo de dato, anteriormente necesitaremos cambiar las comas por puntos, ya que python
# no reconoce las comas en los números, solo los puntos. Para ello aplicaremos un replace sobre cada uno de los elementos de las columnas, 
# y pediremos que nos modifique las comas por puntos.
def buscar_reemplazar(dataframe, columnas):

    """
    Función que cambia los datos numéricos de una columna de un dataframe, que están en tipo string, y contienen comas. 

    Args:
        dataframe (DataFrame): Dataframe que contiene las columnas a modificar
        columnas (lista de columnas del DataFrame): lista de columnas del DataFrame, de las cuales se quiere realizar el cambio de datos.

    Returns:
        DataFrame: dataframe con las columnas indicadas con los datos numéricos cambiados a tipo float.
    """
    for columna in columnas:
    
        dataframe[columna] = dataframe[columna].str.replace(",", ".").astype(float)

    return dataframe

In [37]:
datos_2013 = buscar_reemplazar(datos_2013, columnas_2013)
datos_2013.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,0.0,0.0,1297.13,0.0,31/12/2013,2013


In [38]:
# En estas columnas tenemos información de tipo fecha pero las tenemos en tipo objeto. Vamos a realizar el tipo de cambio de dato.
print(datos_2013["DATA LANÇAMENTO"].dtypes)
print(datos_2013["ANO EXERCÍCIO"].dtypes)

object
int64


In [39]:
# Como queremos modificar las columnas anteriores, nos creamos una lista que contenga las mismas. Para ello hacemos uso de iloc. 
columnas_2013_fecha = datos_2013.iloc[:,14: 16].columns
columnas_2013_fecha

Index(['DATA LANÇAMENTO', 'ANO EXERCÍCIO'], dtype='object')

In [40]:
# Para poder realizar el cambio de dato de str a datetime nos creamos la siguiente función
def cambio_fecha(dataframe, lista_columnas):
    """Función que recibe una lista de columnas de un DataFrame y les cambia el tipo de dato a datetime

    Args:
        dataframe (DataFrame): DataFrame
        lista_columnas (columnas del DataFrame): columnas del DataFrame pasado a la función. 

    Returns:
        DataFrame: devolverá un valor nulo (np.nan) en caso de que se encuentre con un valor nulo,
        o el valor en tipo datetime en el resto de casos. 
    """
    for columna in lista_columnas:
        if columna == None:
            return np.nan
        else:
            dataframe[columna] = pd.to_datetime(dataframe[columna], format = "mixed")
    return dataframe

In [41]:
datos_2013 = cambio_fecha(datos_2013, columnas_2013_fecha)
datos_2013.head()

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,0.0,0.0,1297.13,0.0,2013-12-31,1970-01-01 00:00:00.000002013
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,0.0,0.0,26666621.42,0.0,2013-12-31,1970-01-01 00:00:00.000002013
2,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS E JUROS DE MORA,0.0,0.0,301251.13,0.0,2013-12-31,1970-01-01 00:00:00.000002013
3,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIV.ATIVA POR INFRAÇÃO ADMINISTRATIVA,0.0,0.0,1855.58,0.0,2013-12-31,1970-01-01 00:00:00.000002013
4,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",OUTRAS RESTITUICOES,0.0,0.0,52140.68,0.0,2013-12-31,1970-01-01 00:00:00.000002013


In [42]:
info_df(datos_2013)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,4498,0,0.0
NOME ÓRGÃO SUPERIOR,object,4498,0,0.0
CÓDIGO ÓRGÃO,int64,4498,0,0.0
NOME ÓRGÃO,object,4498,0,0.0
CÓDIGO UNIDADE GESTORA,int64,4498,0,0.0
NOME UNIDADE GESTORA,object,4498,0,0.0
CATEGORIA ECONÔMICA,object,4498,0,0.0
ORIGEM RECEITA,object,4498,0,0.0
ESPÉCIE RECEITA,object,4498,0,0.0
DETALHAMENTO,object,4498,0,0.0


No tenemos valores nulos en ninguna de las columnas del DataFrame de 2013.

In [43]:
numero_duplicados_df_2013 = datos_2013.duplicated().sum()
numero_duplicados_df_2013

np.int64(0)

**Datos 2014**

In [44]:
# Importamos los datos del año 2014
datos_2014 = pd.read_csv("datos/datos-2014.csv", sep = ";", encoding = "latin-1")
datos_2014. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORÁRIOS DE ADVOGADOS,"0,00","0,00","46699948,08","0,00",31/12/2014,2014


In [45]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2014 = datos_2014.copy()
copia_datos_2014.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORÁRIOS DE ADVOGADOS,"0,00","0,00","46699948,08","0,00",31/12/2014,2014


Información general del DataFrame de 2014

In [46]:
info_df(datos_2014)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,4553,0,0.0
NOME ÓRGÃO SUPERIOR,object,4553,0,0.0
CÓDIGO ÓRGÃO,int64,4553,0,0.0
NOME ÓRGÃO,object,4553,0,0.0
CÓDIGO UNIDADE GESTORA,int64,4553,0,0.0
NOME UNIDADE GESTORA,object,4553,0,0.0
CATEGORIA ECONÔMICA,object,4553,0,0.0
ORIGEM RECEITA,object,4553,0,0.0
ESPÉCIE RECEITA,object,4553,0,0.0
DETALHAMENTO,object,4553,0,0.0


No hay valores nulos en el DataFrame de 2014

In [47]:
# Nos encontramos con varias columnas que contienen datos númericos que tienen formato objeto.
 
print(datos_2014["VALOR PREVISTO ATUALIZADO"].dtypes)
print(datos_2014["VALOR LANÇADO"].dtypes)
print(datos_2014["VALOR REALIZADO"].dtypes)
print(datos_2014["PERCENTUAL REALIZADO"].dtypes)

# Para poder trabajar con estas columnas vamos a necesitar cambiar el tipo de dato de las mismas.

object
object
object
object


In [48]:
# Como queremos modificar las columnas anteriores, nos creamos una lista que contenga las mismas. Para ello hacemos uso de iloc. 
columnas_2014 = datos_2014.iloc[:,10: 14].columns
columnas_2014

Index(['VALOR PREVISTO ATUALIZADO', 'VALOR LANÇADO', 'VALOR REALIZADO',
       'PERCENTUAL REALIZADO'],
      dtype='object')

In [49]:
datos_2014 = buscar_reemplazar(datos_2014, columnas_2014)
datos_2014.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORÁRIOS DE ADVOGADOS,0.0,0.0,46699948.08,0.0,31/12/2014,2014


In [50]:
# En estas columnas tenemos información de tipo fecha pero las tenemos en tipo objeto. Vamos a realizar el tipo de cambio de dato.
print(datos_2014["DATA LANÇAMENTO"].dtypes)
print(datos_2014["ANO EXERCÍCIO"].dtypes)

object
int64


In [51]:
# Como queremos modificar las columnas anteriores, nos creamos una lista que contenga las mismas. Para ello hacemos uso de iloc. 
columnas_2014_fecha = datos_2014.iloc[:,14: 16].columns
columnas_2014_fecha

Index(['DATA LANÇAMENTO', 'ANO EXERCÍCIO'], dtype='object')

In [52]:
datos_2014 = cambio_fecha(datos_2014, columnas_2014_fecha)
datos_2014.head()

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORÁRIOS DE ADVOGADOS,0.0,0.0,46699948.08,0.0,2014-12-31,1970-01-01 00:00:00.000002014
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",MULTAS E JUROS PREVISTOS EM CONTRATOS,0.0,0.0,15920.83,0.0,2014-12-31,1970-01-01 00:00:00.000002014
2,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS E JUROS DE MORA,0.0,0.0,2195899.92,0.0,2014-12-31,1970-01-01 00:00:00.000002014
3,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes - intra-orçamentárias,Outras Receitas Correntes,Indenizações e restituições,RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,0.0,0.0,232.02,0.0,2014-12-31,1970-01-01 00:00:00.000002014
4,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receita de Serviços,Receita de Serviços,TAR.INSCR.CONCURSOS E PROCESSOS SELETIVOS,0.0,0.0,367815.01,0.0,2014-12-31,1970-01-01 00:00:00.000002014


In [53]:
numero_duplicados_df_2014 = datos_2014.duplicated().sum()
numero_duplicados_df_2014

np.int64(0)

**Datos 2015**

In [54]:
# Importamos los datos del año 2015
datos_2015 = pd.read_csv("datos/datos-2015.csv", sep = ";", encoding = "latin-1")
datos_2015. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORARIOS DE ADVOGADOS,"0,00","0,00","63829853,40","0,00",31/12/2015,2015


In [55]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2015 = datos_2015.copy()
copia_datos_2015.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORARIOS DE ADVOGADOS,"0,00","0,00","63829853,40","0,00",31/12/2015,2015


Información general del DataFrame de 2014

In [56]:
info_df(datos_2015)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,4523,0,0.0
NOME ÓRGÃO SUPERIOR,object,4523,0,0.0
CÓDIGO ÓRGÃO,int64,4523,0,0.0
NOME ÓRGÃO,object,4523,0,0.0
CÓDIGO UNIDADE GESTORA,int64,4523,0,0.0
NOME UNIDADE GESTORA,object,4523,0,0.0
CATEGORIA ECONÔMICA,object,4523,0,0.0
ORIGEM RECEITA,object,4523,0,0.0
ESPÉCIE RECEITA,object,4523,0,0.0
DETALHAMENTO,object,4523,0,0.0


In [57]:
# Los datos de la columna del monto previsto actualizado contiene números que tienen formato objeto, cuando son datos numéricos. 
print(datos_2014["VALOR PREVISTO ATUALIZADO"].dtypes)
print(datos_2014["VALOR LANÇADO"].dtypes)
print(datos_2014["VALOR REALIZADO"].dtypes)
print(datos_2013["PERCENTUAL REALIZADO"].dtypes)
# Para poder trabajar con estas columnas vamos a necesitar cambiar el tipo de dato de las mismas.

float64
float64
float64
float64


No hay valores nulos en el DataFrame de datos de 2015.

In [58]:
numero_duplicados_df_2015 = datos_2015.duplicated().sum()
numero_duplicados_df_2015

np.int64(0)

**Datos 2016**

In [59]:
# Importamos los datos del año 2016
datos_2016 = pd.read_csv("datos/datos-2016.csv", sep = ";", encoding = "latin-1")
datos_2016. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,ENCARGOS LEGAIS PELA INSCR.EM DIV.ATIVA-PRINC,"0,00","0,00","154948,98","0,00",04/04/2016,2016


In [60]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2016 = datos_2016.copy()
copia_datos_2016.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,ENCARGOS LEGAIS PELA INSCR.EM DIV.ATIVA-PRINC,"0,00","0,00","154948,98","0,00",04/04/2016,2016


Información general sobre los datos de 2016.

In [61]:
info_df(datos_2016)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,194533,0,0.00
NOME ÓRGÃO SUPERIOR,object,194533,0,0.00
CÓDIGO ÓRGÃO,int64,194533,0,0.00
NOME ÓRGÃO,object,194533,0,0.00
CÓDIGO UNIDADE GESTORA,int64,194533,0,0.00
NOME UNIDADE GESTORA,object,194533,0,0.00
CATEGORIA ECONÔMICA,object,194533,0,0.00
ORIGEM RECEITA,object,194533,0,0.00
ESPÉCIE RECEITA,object,194533,0,0.00
DETALHAMENTO,object,194533,0,0.00


Hay valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [62]:
numero_duplicados_df_2016 = datos_2016.duplicated().sum()
numero_duplicados_df_2016

np.int64(0)

In [63]:
# Como queremos modificar las columnas anteriores, nos creamos una lista que contenga las mismas. Para ello hacemos uso de iloc. 
columnas_2016_fecha = datos_2016.iloc[:,14: 16].columns
columnas_2016_fecha

Index(['DATA LANÇAMENTO', 'ANO EXERCÍCIO'], dtype='object')

In [64]:
datos_2016 = cambio_fecha(datos_2016, columnas_2016_fecha)
datos_2016.head(2)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,ENCARGOS LEGAIS PELA INSCR.EM DIV.ATIVA-PRINC,"0,00","0,00","154948,98","0,00",2016-04-04,1970-01-01 00:00:00.000002016
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,ONUS DE SUCUMBENCIA-MULTAS E JUROS,"0,00","0,00","80,84","0,00",2016-08-12,1970-01-01 00:00:00.000002016


In [65]:
info_df(datos_2016)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,194533,0,0.00
NOME ÓRGÃO SUPERIOR,object,194533,0,0.00
CÓDIGO ÓRGÃO,int64,194533,0,0.00
NOME ÓRGÃO,object,194533,0,0.00
CÓDIGO UNIDADE GESTORA,int64,194533,0,0.00
NOME UNIDADE GESTORA,object,194533,0,0.00
CATEGORIA ECONÔMICA,object,194533,0,0.00
ORIGEM RECEITA,object,194533,0,0.00
ESPÉCIE RECEITA,object,194533,0,0.00
DETALHAMENTO,object,194533,0,0.00


**Datos 2017**

In [66]:
# Importamos los datos del año 2017
datos_2017 = pd.read_csv("datos/datos-2017.csv", sep = ";", encoding = "latin-1")
datos_2017. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,OUTRAS RECEITAS-PRIMARIAS-PRINCIPAL,"0,00","0,00","198,00","0,00",29/08/2017,2017


In [67]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2017 = datos_2017.copy()
copia_datos_2017.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,OUTRAS RECEITAS-PRIMARIAS-PRINCIPAL,"0,00","0,00","198,00","0,00",29/08/2017,2017


In [68]:
info_df(datos_2017)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,190479,0,0.00
NOME ÓRGÃO SUPERIOR,object,190479,0,0.00
CÓDIGO ÓRGÃO,int64,190479,0,0.00
NOME ÓRGÃO,object,190479,0,0.00
CÓDIGO UNIDADE GESTORA,int64,190479,0,0.00
NOME UNIDADE GESTORA,object,190479,0,0.00
CATEGORIA ECONÔMICA,object,190479,0,0.00
ORIGEM RECEITA,object,190479,0,0.00
ESPÉCIE RECEITA,object,190479,0,0.00
DETALHAMENTO,object,190479,0,0.00


Hay 86 valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [69]:
numero_duplicados_df_2017 = datos_2017.duplicated().sum()
numero_duplicados_df_2017

np.int64(0)

**Datos 2018**

In [70]:
# Importamos los datos del año 2018
datos_2018 = pd.read_csv("datos/datos-2018.csv", sep = ";", encoding = "latin-1")
datos_2018. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receitas Correntes - a classificar,Receitas Correntes - a classificar,Receitas Correntes - a classificar,"0,00","0,00","-1693,72","0,00",25/07/2018,2018


In [71]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2018 = datos_2018.copy()
copia_datos_2018.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receitas Correntes - a classificar,Receitas Correntes - a classificar,Receitas Correntes - a classificar,"0,00","0,00","-1693,72","0,00",25/07/2018,2018


In [72]:
info_df(datos_2018)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,173944,0,0.00
NOME ÓRGÃO SUPERIOR,object,173944,0,0.00
CÓDIGO ÓRGÃO,int64,173944,0,0.00
NOME ÓRGÃO,object,173944,0,0.00
CÓDIGO UNIDADE GESTORA,int64,173944,0,0.00
NOME UNIDADE GESTORA,object,173944,0,0.00
CATEGORIA ECONÔMICA,object,173944,0,0.00
ORIGEM RECEITA,object,173944,0,0.00
ESPÉCIE RECEITA,object,173944,0,0.00
DETALHAMENTO,object,173944,0,0.00


Hay 115 valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [73]:
numero_duplicados_df_2018 = datos_2018.duplicated().sum()
numero_duplicados_df_2018

np.int64(0)

**Datos 2019**

In [74]:
# Importamos los datos del año 2019
datos_2019 = pd.read_csv("datos/datos-2019.csv", sep = ";", encoding = "latin-1")
datos_2019. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receita de Serviços,Serviços Administrativos e Comerciais Gerais,INSCR.EM CONCURSOS E PROC.SELETIVOS-PRINCIPAL,"0,00","0,00","-95,00","0,00",12/06/2019,2019


In [75]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2019 = datos_2019.copy()
copia_datos_2019.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receita de Serviços,Serviços Administrativos e Comerciais Gerais,INSCR.EM CONCURSOS E PROC.SELETIVOS-PRINCIPAL,"0,00","0,00","-95,00","0,00",12/06/2019,2019


In [76]:
info_df(datos_2019)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,176828,0,0.00
NOME ÓRGÃO SUPERIOR,object,176828,0,0.00
CÓDIGO ÓRGÃO,int64,176828,0,0.00
NOME ÓRGÃO,object,176828,0,0.00
CÓDIGO UNIDADE GESTORA,int64,176828,0,0.00
NOME UNIDADE GESTORA,object,176828,0,0.00
CATEGORIA ECONÔMICA,object,176828,0,0.00
ORIGEM RECEITA,object,176828,0,0.00
ESPÉCIE RECEITA,object,176828,0,0.00
DETALHAMENTO,object,176828,0,0.00


Hay 86 valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [77]:
numero_duplicados_df_2019 = datos_2019.duplicated().sum()
numero_duplicados_df_2019

np.int64(0)

**Datos 2020**

In [78]:
# Importamos los datos del año 2019
datos_2020 = pd.read_csv("datos/datos-2020.csv", sep = ";", encoding = "latin-1")
datos_2020. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RESTITUIÇÃO DE CONVÊNIOS-PRIMÁRIAS-DÍV.ATIVA,"0,00","0,00","5516,90","0,00",28/04/2020,2020


In [79]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2020 = datos_2020.copy()
copia_datos_2020.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RESTITUIÇÃO DE CONVÊNIOS-PRIMÁRIAS-DÍV.ATIVA,"0,00","0,00","5516,90","0,00",28/04/2020,2020


In [80]:
info_df(datos_2020)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,142348,0,0.00
NOME ÓRGÃO SUPERIOR,object,142348,0,0.00
CÓDIGO ÓRGÃO,int64,142348,0,0.00
NOME ÓRGÃO,object,142348,0,0.00
CÓDIGO UNIDADE GESTORA,int64,142348,0,0.00
NOME UNIDADE GESTORA,object,142348,0,0.00
CATEGORIA ECONÔMICA,object,142348,0,0.00
ORIGEM RECEITA,object,142348,0,0.00
ESPÉCIE RECEITA,object,142348,0,0.00
DETALHAMENTO,object,142348,0,0.00


Hay 86 valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [81]:
numero_duplicados_df_2020 = datos_2020.duplicated().sum()
numero_duplicados_df_2020

np.int64(0)

**Datos 2021**

In [82]:
# Importamos los datos del año 2019
datos_2021 = pd.read_csv("datos/datos-2021.csv", sep = ";", encoding = "latin-1")
datos_2021. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas de Capital,Operações de Crédito,Operações de crédito - mercado interno,TITULOS DE RESPONS.TES.NAC.-MERC.INT.-PRINC.,"2224034904,00","0,00","0,00","0,00",23/04/2021,2021


In [83]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2021 = datos_2021.copy()
copia_datos_2021.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas de Capital,Operações de Crédito,Operações de crédito - mercado interno,TITULOS DE RESPONS.TES.NAC.-MERC.INT.-PRINC.,"2224034904,00","0,00","0,00","0,00",23/04/2021,2021


In [84]:
info_df(datos_2021)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,134593,0,0.00
NOME ÓRGÃO SUPERIOR,object,134593,0,0.00
CÓDIGO ÓRGÃO,int64,134593,0,0.00
NOME ÓRGÃO,object,134593,0,0.00
CÓDIGO UNIDADE GESTORA,int64,134593,0,0.00
NOME UNIDADE GESTORA,object,134593,0,0.00
CATEGORIA ECONÔMICA,object,134593,0,0.00
ORIGEM RECEITA,object,134593,0,0.00
ESPÉCIE RECEITA,object,134593,0,0.00
DETALHAMENTO,object,134593,0,0.00


Hay 123 valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [85]:
numero_duplicados_df_2021 = datos_2021.duplicated().sum()
numero_duplicados_df_2021

np.int64(0)